In [ ]:
from scipy.io import wavfile
import glob
import cv2
from scipy.fftpack import fft,fftfreq
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import pylab
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import librosa    
import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras import optimizers
import keras

%matplotlib inline


In [ ]:
audio_data = []
labels = []
sampling_rate = []
file_names = []


In [ ]:
samplerate, data = wavfile.read("dataset/dog/1-30344-A.wav")
data.shape


In [ ]:
time_sec = (len(data)/samplerate)
step = time_sec/len(data)
print(time_sec)
print(step)
i=0
time_divion=[]
while i<=time_sec-step:
    
    time_divion.append(i)
    i=i+step
# the fourth second step
four_sec_step_number = (4*len(time_divion))/time_sec
print(four_sec_step_number)

In [ ]:
label_number=0
classes = []
audio_data = []
labels = []
sampling_rate = []
file_names = []
data = []
noisy_removed=[]
noise=[]
for filepath in glob.iglob('dataset/*'):
    
    #print(filepath[9:])
    
    #print(filepath)
    classes.append(filepath[8:])

print(classes)

for i in classes:
    print("the class = "+i+", the label = "+str(label_number))
    for j in glob.iglob('dataset/'+i+'/*'):
        s, y = wavfile.read(j)
        #y, s = librosa.load(j, sr=44000) # Downsample 44.1kHz to 8kHz
        #reduced_noise = nr.reduce_noise(audio_clip=y, noise_clip=y, verbose=False)
        #print(s)
        #print(j)
        data.append([y[:,0],label_number])
        #noise.append(y)
        #labels.append(label_number)
        
    label_number = label_number + 1
print(len(labels))
#print(labels)


In [ ]:
import random

random.shuffle(data)
audio_data=[]
labels=[]
for i,j in data:
    audio_data.append(i)
    labels.append(j)
    


In [ ]:
example = random.randint(0,400)


In [ ]:
save_path='rain_before.jpg'
plt.plot(time_divion[0:192000],audio_data[example][0:192000])
#plt.show()
plt.title('signal in real time')
pylab.savefig(save_path, bbox_inches=None, pad_inches=0)
pylab.close()


In [ ]:
labels[example]

In [ ]:
datafft = fft(audio_data[example][0:192000])
fft_sample = fft(audio_data[example][0:192000]) # take the fourier transform 

fftabs = abs(fft_sample)
print(fftabs.shape)
fftabs=fftabs/np.maximum.reduce(fftabs)
freqs = fftfreq(len(audio_data[example][0:192000]),1/48000)
freqs=freqs/np.maximum.reduce(freqs)

##print(len(freqs))
plt.plot(freqs[:len(fftabs)//2],fftabs[:len(fftabs)//2])
#plt.plot(freqs,fftabs)
#plt.show()

save_path = 'rain.jpg'

#plt.axis('off') # no axis
plt.title('Fourier Transform')
pylab.savefig(save_path, bbox_inches=None, pad_inches=0)
pylab.close()

In [ ]:
fourier_transform=[]
for i in range(0,len(audio_data)):
    
    fast_f = fft(audio_data[i][0:192000]) # take the fourier transform 
    fft_abs = abs(fast_f)
    fft_abs=fft_abs/np.maximum.reduce(fft_abs)
    freqs = fftfreq(len(audio_data[i][0:192000]),1/48000)
    freqs=freqs/np.maximum.reduce(freqs)
    print(freqs.shape)
    fourier_transform.append([fft_abs[:len(fft_abs)//2],freqs[:len(fft_abs)//2]])
    



In [ ]:
print(len(audio_data))
print(len(fourier_transform))
#print(len(audio_data))
fourier_transform=np.array(fourier_transform)
print(fourier_transform.shape)

In [ ]:
fourier_transform=fourier_transform.reshape(400,-1)
fourier_transform.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
data = fourier_transform
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(fourier_transform)


print(np.amax(fourier_transform))

normalized_features = scaler.transform(fourier_transform)
print(np.amax(normalized_features))

In [ ]:
from sklearn import tree
X_train, X_test, y_train, y_test = train_test_split(fourier_transform, labels, test_size=0.20, random_state=1150)
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
features_convolution = np.reshape(normalized_features,(400,1000, -1,1))

In [ ]:
y=keras.utils.to_categorical(labels, num_classes=10, dtype='float32')


In [ ]:
model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape=features_convolution.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#'''
#model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Dropout(0.2))

#'''
#'''
model.add(Conv2D(64, (3, 3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#'''


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

#model.add(Dense(1000))#input_shape=features.shape[1:]
model.add(Dense(64))#input_shape=features.shape[1:]

model.add(Dense(10))
model.add(Activation('softmax'))
sgd = optimizers.SGD(lr=0.0000001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(features_convolution, y,batch_size=1, epochs=40,validation_split=0.2)